<a href="https://colab.research.google.com/github/fboldt/aulasml/blob/master/aula6_arvores_atributos_discretos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
urllib.request.urlretrieve(url, 'car.data')

('car.data', <http.client.HTTPMessage at 0x7f64625b6790>)

In [ ]:
import numpy as np
data = np.loadtxt('car.data', dtype='str', delimiter=',')
X, y = data[:,:-1], data[:,-1]
X.shape, y.shape

((1728, 6), (1728,))

In [ ]:
combinações = 1
for i in range(X.shape[1]):
  valores = set(X[:,i])
  print(valores)
  combinações *= len(valores)
print(combinações)

{'low', 'med', 'high', 'vhigh'}
{'low', 'med', 'high', 'vhigh'}
{'5more', '3', '4', '2'}
{'more', '4', '2'}
{'small', 'med', 'big'}
{'med', 'high', 'low'}
1728


In [ ]:

X[550:555,:], y[550:555]

(array([['high', 'high', '2', '4', 'small', 'med'],
        ['high', 'high', '2', '4', 'small', 'high'],
        ['high', 'high', '2', '4', 'med', 'low'],
        ['high', 'high', '2', '4', 'med', 'med'],
        ['high', 'high', '2', '4', 'med', 'high']], dtype='<U5'),
 array(['unacc', 'acc', 'unacc', 'unacc', 'acc'], dtype='<U5'))

In [ ]:
from collections import Counter
from sklearn.metrics import accuracy_score

def maisFrequente(y):
  return Counter(y.flat).most_common(1)[0][0]

class ZeroR():
  def fit(self, X, y):
    self.resposta = maisFrequente(y)
  def predict(self, X):
    y = np.empty((X.shape[0]), dtype='<U5')
    y[:] = self.resposta
    return y

modelo = ZeroR()
modelo.fit(X, y)
ypred = modelo.predict(X)
accuracy_score(y, ypred), (y == ypred)

(0.7002314814814815, array([ True,  True,  True, ...,  True, False, False]))

In [ ]:
def impureza(y): #Gini
  labels = list(set(y))
  labels.sort()
  probabilidades = np.zeros((len(labels),))
  for i,k in enumerate(labels):
    probabilidades[i] = sum(y==k)/len(y)
  result = 1 - sum(probabilidades**2)
  return result

ytmp = y[:]
valor = impureza(ytmp)
valor

0.457283763074417

In [ ]:
def impurezaValor(x, y, valor):
  iguais = x==valor
  impurezaIguais = impureza(y[iguais])
  propIguais = sum(iguais)/len(y)
  impurezaDiferentes = impureza(y[~iguais])
  propDiferentes = sum(~iguais)/len(y)
  impurezaTotal = propIguais*impurezaIguais + propDiferentes*impurezaDiferentes
  return impurezaTotal

impurezaValor(X[:,0], y, 'vhigh')

0.44934645776177407

In [ ]:
def impurezaMínima(X, y):
  impurezas = []
  característicasValores = []
  for i in range(X.shape[1]):
    valores = sorted(list(set(X[:,i])))
    for valor in valores:
      característicasValores.append([i,valor])
      impurezasValorCaracterística = impurezaValor(X[:,i], y, valor)
      impurezas.append(impurezasValorCaracterística)
  característicasValores = np.array(característicasValores)
  impurezas = np.array(impurezas)
  menorImpureza = np.argmin(impurezas)
  característica, valor = característicasValores[menorImpureza]
  return impurezas[menorImpureza], int(característica), valor

impurezaMínima(X, y)

(0.38615712609310704, 3, '2')

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class Arvore(BaseEstimator, ClassifierMixin):
  
  def fit(self, X, y):
    self.impureza, self.característica, self.valor = impurezaMínima(X, y)
    iguais = X[:,self.característica] == self.valor
    if sum(iguais)>0 and sum(~iguais)>0:
      self.iguais = Arvore()
      self.iguais.fit(X[iguais,:],y[iguais])
      self.diferentes = Arvore()
      self.diferentes.fit(X[~iguais,:],y[~iguais])
    else:
      self.resposta = maisFrequente(y)

  def predict(self, X):
    y = np.empty((X.shape[0]), dtype='<U5')
    if hasattr(self, 'resposta'):
      y[:] = self.resposta
    else:
      iguais = X[:,self.característica] == self.valor
      y[iguais] = self.iguais.predict(X[iguais,:])
      y[~iguais] = self.diferentes.predict(X[~iguais,:])
    return y

modelo = Arvore()
modelo.fit(X, y)
ypred = modelo.predict(X)
accuracy_score(y, ypred), (y == ypred)

(1.0, array([ True,  True,  True, ...,  True,  True,  True]))

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(Arvore(), X, y)
scores['test_score'], np.mean(scores['test_score'])

(array([0.62716763, 0.73121387, 0.75144509, 0.75362319, 0.8057971 ]),
 0.7338493758900897)